In [2]:
import numpy as np
import csv
import pandas as pd
import os
import matplotlib.pyplot as plt
from polyfuzz import PolyFuzz

import productmatching


PATH = os.getcwd()

/Users/hassansaeed/Documents/Anaconda/anaconda3/envs/my/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Read all-merged and matches file

In [3]:
base_folder = 'aws/dev-dashy-refined-datasets/'
df_all_merged = productmatching.read_file_from_csv(base_folder + "analytics/products", "all-merged.csv")
""" df_roundeye = productmatching.read_file_from_csv(base_folder + "datasets", "RoundEye-merged.csv")
df_foodservice = productmatching.read_file_from_csv(base_folder + "datasets", "FoodServicesDirect-merged.csv")
df_birite = productmatching.read_file_from_csv(base_folder + "datasets", "BiRite-merged.csv")
df_warehouse= productmatching.read_file_from_csv(base_folder + "datasets", "115WareHouse-merged.csv")
df_matches = productmatching.read_file_from_csv(base_folder + "matching", "matches.csv")
df_inv = productmatching.read_file_from_csv(base_folder + "datasets", "invoices-unique-products.csv") """


/var/folders/rj/rrq458bj1mb9fpw62rmd9yb80000gn/T/ipykernel_52121/355239101.py:2: DtypeWarning: Columns (17,18,19,22) have mixed types.Specify dtype option on import or set low_memory=False.
  df_all_merged = productmatching.read_file_from_csv(base_folder + "analytics/products", "all-merged.csv")


read 68136  rows from aws/dev-dashy-refined-datasets/analytics/products all-merged.csv


' df_roundeye = productmatching.read_file_from_csv(base_folder + "datasets", "RoundEye-merged.csv")\ndf_foodservice = productmatching.read_file_from_csv(base_folder + "datasets", "FoodServicesDirect-merged.csv")\ndf_birite = productmatching.read_file_from_csv(base_folder + "datasets", "BiRite-merged.csv")\ndf_warehouse= productmatching.read_file_from_csv(base_folder + "datasets", "115WareHouse-merged.csv")\ndf_matches = productmatching.read_file_from_csv(base_folder + "matching", "matches.csv")\ndf_inv = productmatching.read_file_from_csv(base_folder + "datasets", "invoices-unique-products.csv") '

In [ ]:
#Get all hashes from all_merged
all_merged_hash = df_all_merged['hash'].to_list()
#Dedupe if any
all_merged_hash = list(set(all_merged_hash))

#Get all source hash from matches
matches_source_hash = df_matches['source_hash'].to_list()
#Dedupe if any
matches_source_hash = list(set(matches_source_hash))

#Get all destination hash from matches
matches_dest_hash = df_matches['dest_hash'].to_list()
#Dedupe if any
matches_dest_hash = list(set(matches_dest_hash))

# common codes
#intersection_source = list(set(all_merged_hash).intersection(matches_source_hash))
#print(len(intersection_source), len(matches_source_hash))
#intersection_dest = list(set(all_merged_hash).intersection(matches_dest_hash))
#print(len(intersection_dest), len(matches_dest_hash))

# source hashes missing from all merged
missing_source = list(set(matches_source_hash).difference(all_merged_hash))
print(len(missing_source), 'source hashes missing from all_merged')
missing_dest = list(set(matches_dest_hash).difference(all_merged_hash))
print(len(missing_dest), 'destination hashes missing from all_merged')

# See 10 missing source hashes
missing_source[:10]


In [14]:
#df_matches[df_matches['source_hash'].isin(missing_source[:10])]
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

#df_matches[df_matches['source_hash'].isin(missing_source) & (df_matches['notes'] != 'missing')].groupby(['source_file']).size().reset_index(name='counts')
def foo(x):
    if x in ['Food Service Direct', 'birite', 'Webstaurant Store', 'Warehouse 115'\
        ,'Round Eye Supply', 'Chefs Store' ]:
        return x
    else:
        return 'Others/Invoice'
df_all_merged['type'] = df_all_merged['supplier'].apply(foo)
_1 = df_all_merged.groupby(['type']).size().reset_index(name='total_counts').sort_values(['total_counts'],ascending=False)
_2 = df_all_merged[(df_all_merged['UPC'] == '')].groupby(['type']).size().reset_index(name='missing_counts').sort_values(['missing_counts'],ascending=False)
_1 = _1.merge(_2,how='inner',left_on=['type'],right_on=['type'])
_1['missing_%'] = 100*_1['missing_counts']/_1['total_counts']
_1

,type,total_counts,missing_counts,missing_%
0,Food Service Direct,16420,1756,10.694275
1,birite,13690,8304,60.657414
2,Webstaurant Store,13399,1626,12.135234
3,Warehouse 115,9880,41,0.414980
4,Round Eye Supply,9870,9870,100.000000
5,Others/Invoice,3875,3875,100.000000
6,Chefs Store,1002,1002,100.000000


,type,counts
3,Round Eye Supply,9870
6,birite,8304
2,Others/Invoice,3875
1,Food Service Direct,1756
5,Webstaurant Store,1626
0,Chefs Store,1002
4,Warehouse 115,41


In [ ]:
# Change source file to see product details for a specific file
df_matches[(df_matches['source_hash'].isin(missing_source)) & \
    (df_matches['notes'] != 'missing') & (df_matches['source_file'] == 'BiRite-merged.csv')][:2]

In [ ]:
# Check product details in all_merged
df_all_merged[df_all_merged['name'] == 'BASE CHICKEN GRANULES NO MSG']


In [ ]:
# Check product details in individual file
df_birite[df_birite['name'] == 'BASE CHICKEN GRANULES NO MSG']



In [ ]:
#df_roundeye[df_roundeye['name'] == '44 LB Thai Jasmine Rice']
#df_foodservice[df_foodservice['name'] == 'Coffee Pro 30-Cup Percolating Urn, Stainless Steel']
#df_birite[df_birite['name'] == 'BASE CHICKEN GRANULES NO MSG GLUTENFREE']
#df_all_merged[df_all_merged['name'] == 'Maggi Gluten Free Chicken Base (No Added MSG) 25lb.']
import hashlib

x = 'biriteBASE CHICKEN GRANULES NO MSG GLUTENFREEUPC-07482611303943619'
y = 'biriteBASE CHICKEN GRANULES NO MSG GLUTENFREEUPC-07482611303943619.0'

print(hashlib.md5(x.encode()).hexdigest(), hashlib.md5(y.encode()).hexdigest())

x = 'biriteSAUCE HOT GREEN CHILI PEPPER GLASSUPC-049733950118133818'
y = 'biriteSAUCE HOT GREEN CHILI PEPPER GLASSUPC-049733950118133818.0'
print(hashlib.md5(x.encode()).hexdigest(), hashlib.md5(y.encode()).hexdigest())
